In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import json

%matplotlib inline

# Analysis of results

## Import of data

In [19]:
DATE = "2020-12-16"


def import_native_and_bytecode(filename):
    native = []
    bytecode = []
    with open(filename) as f:
        for line in f:
            entry = json.loads(line)
            name_sections = list(entry['name'].split('.'))
            if name_sections[0].endswith('bytecode'):
                name_sections[0] = name_sections[0][:-9]
                entry['name'] = '.'.join(name_sections)
                bytecode.append(entry)
            else:
                native.append(entry)
    return pd.json_normalize(native), pd.json_normalize(bytecode)


def import_just_bytecode(filename):
    data = []
    for line in open(filename):
        entry = json.loads(line)
        name_sections = list(entry['name'].split('.'))
        name_sections[0] = name_sections[0][:-9]
        entry['name'] = '.'.join(name_sections)
        data.append(entry)
    return pd.json_normalize(data)
            
    
native, stock = import_native_and_bytecode(f'{DATE}/4.11.1+stock.orun.bench')
compile_only = import_just_bytecode(f'{DATE}/4.11.1+jit+compileonly.orun.bench')
jit_disabled = import_just_bytecode(f'{DATE}/4.11.1+jit+disabled.orun.bench')
jit = import_just_bytecode(f'{DATE}/4.11.1+jit.orun.bench')

In [42]:
compile_only

,name,command,time_secs,user_time_secs,sys_time_secs,maxrss_kB,codesize,ocaml_url,ocaml.version,ocaml.c_compiler,...,gc.allocated_words,gc.minor_words,gc.promoted_words,gc.major_words,gc.minor_collections,gc.major_collections,gc.heap_words,gc.heap_chunks,gc.top_heap_words,gc.compactions
0,naive-multilayer.,taskset --cpu-list 5 ./naive_multilayer.bc,19.598620,19.592298,0.000000,11136,0.0,../../src,4.11.1+dev_jit,gcc,...,3908428073,3908412655,3278991,3294409,15001,184,126976,1,126976,0
1,lists.record_1,taskset --cpu-list 5 ./lists.bc record 1,0.804723,0.804292,0.000000,9716,0.0,../../src,4.11.1+dev_jit,gcc,...,65000798,65000515,1585,1868,249,4,61440,1,188416,2
2,imrin_mem_rw.10_000_50_000_20_100_000_000,taskset --cpu-list 5 ./irmin_mem_rw.bc 10_000 ...,31.978998,31.961465,0.006664,17044,0.0,../../src,4.11.1+dev_jit,gcc,...,4761174129,4761149266,792288,817151,18167,9,661504,9,661504,0
3,lists.float-tuple_100000,taskset --cpu-list 5 ./lists.bc float-tuple 10...,0.651501,0.644318,0.006678,33968,0.0,../../src,4.11.1+dev_jit,gcc,...,51000888,51000605,44732389,44732672,220,74,1742336,17,2651136,14
4,nbody.50_000_000,taskset --cpu-list 5 ./nbody.bc 50_000_000,109.805796,109.753779,0.003332,11236,0.0,../../src,4.11.1+dev_jit,gcc,...,53000009642,53000005923,1715362,1719081,202385,415,61440,1,188416,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,weakretain.75_10000000,taskset --cpu-list 5 ./weakretain.bc 75 10000000,1.622576,1.605255,0.016637,89636,0.0,../../src,4.11.1+dev_jit,gcc,...,75000057,64999231,2146,10002972,249,2,18127360,2,18127360,0
179,lists.int-array_100000,taskset --cpu-list 5 ./lists.bc int-array 100000,0.564185,0.553719,0.010001,32668,0.0,../../src,4.11.1+dev_jit,gcc,...,45900888,45900605,39715703,39715986,200,65,1737216,15,2297856,12
180,lazylist.1000_10000,taskset --cpu-list 5 ./lazylist.bc 1000 10000,0.187534,0.183897,0.003344,10336,0.0,../../src,4.11.1+dev_jit,gcc,...,80059718,80059227,1551019,1551510,361,113,61440,1,188416,2
181,LU_decomposition.1024,taskset --cpu-list 5 ./LU_decomposition.bc 1024,29.523704,29.511287,0.003332,43216,0.0,../../src,4.11.1+dev_jit,gcc,...,2883235061,2879040030,39909,4234940,11003,39,7677952,5,7677952,0


In [45]:
jit.sort_values(by='time_secs', ascending=False)

,name,command,time_secs,user_time_secs,sys_time_secs,maxrss_kB,codesize,ocaml_url,ocaml.version,ocaml.c_compiler,...,gc.allocated_words,gc.minor_words,gc.promoted_words,gc.major_words,gc.minor_collections,gc.major_collections,gc.heap_words,gc.heap_chunks,gc.top_heap_words,gc.compactions
10,fannkuchredux.12,taskset --cpu-list 5 ./fannkuchredux.bc 12,939.428541,939.202396,0.000000,11284,0.0,../../src,4.11.1+dev_jit,gcc,...,10391898334,10391894744,83377,86967,39652,23,61440,1,188416,2
175,fannkuchredux2.12,taskset --cpu-list 5 ./fannkuchredux2.bc 12,732.539329,732.406358,0.000000,8920,0.0,../../src,4.11.1+dev_jit,gcc,...,8192,4602,0,3590,0,0,126976,1,126976,0
163,mandelbrot6.16_000,taskset --cpu-list 5 ./mandelbrot6.bc 16_000,405.815695,405.704455,0.000000,11356,0.0,../../src,4.11.1+dev_jit,gcc,...,103041862887,103041859280,8824476,8828083,394122,2099,61440,1,188416,2
130,evolutionary_algorithm.10000_10000,taskset --cpu-list 5 ./evolutionary_algorithm....,293.655132,292.936449,0.629782,2815316,0.0,../../src,4.11.1+dev_jit,gcc,...,17030544482,15730282341,1185440,1301447581,60029,32,357150208,54,357150208,0
56,knucleotide.,taskset --cpu-list 5 ./knucleotide.bc,225.617750,225.439937,0.116625,411128,0.0,../../src,4.11.1+dev_jit,gcc,...,1891491094,1842832767,2684306,51342633,7036,12,87480320,11,87480320,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,lists.int_10000,taskset --cpu-list 5 ./lists.bc int 10000,0.104030,0.103805,0.000000,11164,0.0,../../src,4.11.1+dev_jit,gcc,...,15031787,15031504,980973,981256,66,20,122880,2,188416,3
6,kernel2.3_1,taskset --cpu-list 5 ./kernel2.bc 3 1,0.002379,0.000000,0.002160,8080,0.0,../../src,4.11.1+dev_jit,gcc,...,10144,7778,0,2366,0,0,126976,1,126976,0
100,kernel3.3_1_1,taskset --cpu-list 5 ./kernel3.bc 3 1 1,0.002307,0.002048,0.000000,8252,0.0,../../src,4.11.1+dev_jit,gcc,...,10926,8548,0,2378,0,0,126976,1,126976,0
13,bytes_bench.bytes_set_1000000000,taskset --cpu-list 5 ./bytes_bench.bc bytes_se...,0.002014,0.001794,0.000000,7708,0.0,../../src,4.11.1+dev_jit,gcc,...,1709,1151,0,558,0,0,126976,1,126976,0


In [40]:
def add_suffix(df, suffix):
    return df.add_suffix(suffix).rename(columns={f'name{suffix}': 'name'})

combined = (
    add_suffix(native, '_native')
    .merge(add_suffix(stock, '_stock'), on='name')
    .merge(add_suffix(jit_disabled, '_jit_disabled'), on='name')
    .merge(add_suffix(compile_only, '_compile_only'), on='name')
    .merge(add_suffix(jit, '_jit'), on='name')
)[['name', 'time_secs_native', 'time_secs_stock', 'time_secs_jit_disabled', 'time_secs_compile_only', 'time_secs_jit']]
# combined['speedup'] = combined.time_secs_bytecode / combined.time_secs_native
# combined = combined.sort_values(by='speedup', ascending=False).reset_index(drop=True)
print(combined.to_string())

                                                     name  time_secs_native  time_secs_stock  time_secs_jit_disabled  time_secs_compile_only  time_secs_jit
0                                     lists.float-array_1          0.036676         0.220350                0.337144                0.337327       0.334971
1                                      lists.string_10000          0.321332         0.365521                0.408982                0.407611       0.419782
2                                        stress.100000_75          0.457338         2.113008                3.195530                3.082079       3.083343
3             hashtbl_bench.hashtbl_add_duplicate_2000000          1.341473         1.658269                2.029060                1.958997       2.026095
4                                       naive-multilayer.          1.679385        12.877373               19.398529               19.598620      19.459298
5                                zarith_fact.40_1_000_000       

In [31]:
combined['speedup'].mean()

5.230522939611381